In [1]:
import sys
sys.path.insert(0, '/Users/alam/code/vfftn/src')
from models2 import heisenberg_ham, kronecker_pad, xy_ham 
from mpo import pauli
import numpy as np 
from scipy.special import comb
from functools import reduce
from scipy.optimize import minimize, linear_sum_assignment, dual_annealing
from itertools import permutations, combinations
from tqdm.notebook import tqdm 
import time 

# xxx model
xxx_ham = lambda n : heisenberg_ham(n, jx=1/4,jy=1/4,jz=1/4,h=0)  
xxz_ham = lambda n : heisenberg_ham(n, jx=1/4,jy=1/4,jz=1,h=0)

In [2]:
def total_sz(n): 
    total_z = np.zeros((2**n,2**n), dtype=complex)
    for i in range(n): 
        total_z += kronecker_pad(pauli[3], n, i)
    return total_z

def total_spin(n): 
    total_x = np.zeros((2**n,2**n), dtype=complex)
    total_y = np.zeros((2**n,2**n), dtype=complex)
    total_z = np.zeros((2**n,2**n), dtype=complex)
    for i in range(n):
        total_x += kronecker_pad(pauli[1], n, i) 
        total_y += kronecker_pad(pauli[2], n, i)
        total_z += kronecker_pad(pauli[3], n, i)

    return [total_x, total_y, total_z]

def total_spin_squared(n):
    total_x, total_y, total_z = total_spin(n)
    return total_x @ total_x + total_y @ total_y + total_z @ total_z

def fixed_sz_basis(n, f):
    if f < 0 or f > n:
        return []
    
    result = []
    for positions in combinations(range(n), f):
        binary = ['0'] * n
        for pos in positions:
            binary[pos] = '1'
        result.append(''.join(binary))
    return result

def bin_to_vec(b): 
    vec = np.zeros(2**len(b))
    vec[int(b,2)] = 1.0
    return vec

def u1_projector(n,m):
    basis = fixed_sz_basis(n,m) 
    return np.array([bin_to_vec(b) for b in basis])

def su2_multiplicity(n,j):
    """ returns the multiplicity of the j-th irrep of n tensor products of su(2) """
    return int(comb(n,int(n/2-j)) - comb(n, int(n/2-j)-1)) 

def su2_dimension(j): 
    return int(2*j + 1)  

def su2_projector(n,j,m): 
    """ returns the projector onto the subspace with total spin j and total z-component m """
    U = np.loadtxt(f'N={n}.txt', delimiter=',')
    
    j_cur = 0 if n % 2 == 0 else 1/2
    sector_start = 0 
    while j_cur < j: 
        sector_start += su2_multiplicity(n,j_cur) * su2_dimension(j_cur)
        j_cur += 1

    m_offset = int((-m * 2 + 2*j)/2) if (m*2)%2 == 1 else (-m+j)
    indices = []
    for i in range(su2_multiplicity(n,j)):
        indices.append(sector_start + i*su2_dimension(j) + m_offset)

    P = np.zeros((2**n,2**n), dtype=complex)
    for index in indices: 
        P += np.outer(U[:,index], U[:,index].conj())
    return P, U[:,indices] 

def nn_ansatz(params, n): 
    """ len(params) = 2n """
    ansatz = np.zeros((2**n,2**n), dtype=complex)

    for i in range(n):
        ansatz += params[i] * kronecker_pad(pauli[3], n, i) 

    ZZ = np.kron(pauli[3], pauli[3])
    for i in range(n-1):
        ansatz += params[n+i] * kronecker_pad(ZZ, n, i)
        
    ansatz += params[-1] * np.eye(2**n)
    return ansatz 

def one_local_ansatz(params, n):
    """ len(params) = n + 1 """
    ansatz = np.zeros((2**n,2**n), dtype=complex)

    for i in range(n):
        ansatz += params[i] * kronecker_pad(pauli[3], n, i) 

    ansatz += params[-1] * np.eye(2**n)
    return ansatz

def two_local_ansatz(params, n): 
    """ len(params) = n + n(n-1)/2 + 1 """
    ansatz = np.zeros((2**n,2**n), dtype=complex)

    for i in range(n):
        ansatz += params[i] * kronecker_pad(pauli[3], n, i) 

    # generates pairs of sites from 0 to n-1
    sites = [(i,j) for i in range(n) for j in range(i+1,n)]
    for param_count, (i,j) in enumerate(sites): 
        kron_list = [pauli[3] if k == i or k == j else np.eye(2) for k in range(n)]
        ansatz += params[n + param_count] * reduce(np.kron, kron_list)

    ansatz += params[-1] * np.eye(2**n)
    return ansatz 

def unitary_equivalence(D, H):
    """
    Finds a unitary matrix U such that U D U^† = H. D and H are normal matrices with the same eigenvalues. 
    """
    # Perform eigenvalue decomposition for D and H
    eigvals_D, V_D = np.linalg.eigh(D)  # eigenvalues and eigenvectors of D
    eigvals_H, V_H = np.linalg.eigh(H)  # eigenvalues and eigenvectors of H
    
    # Ensure eigenvalues are sorted consistently (since they might not be in order)
    idx_D = np.argsort(eigvals_D)
    idx_H = np.argsort(eigvals_H)
    
    V_D = V_D[:, idx_D]
    V_H = V_H[:, idx_H]
    
    # Construct U
    U = V_H @ V_D.conj().T
    
    return U

def nn_cost(params, n, target_evs):
    ansatz = basis.conj().T @ nn_ansatz(params, n) @ basis
    ansatz_evals = np.linalg.eigvalsh(ansatz)
    return np.linalg.norm(ansatz_evals - target_evs)

def two_local_cost(params, n, target_evs): 
    ansatz = basis.conj().T @ two_local_ansatz(params, n) @ basis
    ansatz_evals = np.linalg.eigvalsh(ansatz)
    return np.linalg.norm(ansatz_evals - target_evs)

def generate_cost_matrix(params, target, n, ansatz_func=nn_ansatz):
    ansatz = np.diag(ansatz_func(params, n))
    cost_function = lambda a, t: abs(a - t)
    cost_matrix = np.array([[cost_function(a, t) for t in target] for a in ansatz])
    
    if len(ansatz) > len(target):
        dummy_columns = np.zeros((len(ansatz), len(ansatz) - len(target)))
        cost_matrix = np.hstack((cost_matrix, dummy_columns))

    return cost_matrix

def hungarian_cost(params, target, n, ansatz_func=nn_ansatz, quiet=True):
    cost_matrix = generate_cost_matrix(params, target, n, ansatz_func)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    min_cost = cost_matrix[row_ind, col_ind].sum()

    global last_print_time
    if time.time() - last_print_time >= 5:
        print(f"Cost: {min_cost}")
        last_print_time = time.time()

    if not quiet: 
        print("Optimal Assignments:")
        for worker, task in zip(row_ind, col_ind):
            if task < len(target): 
                print(f"Worker {worker} → Task {task} (Cost: {cost_matrix[worker, task]})")
        print(f"\nTotal Minimum Cost: {min_cost}") 

    return min_cost

# Tests

## XY model

In [52]:
# testing one-local ansatz with Hungarian algorithm + simulated annealing
n = 6
ham = xy_ham(n)
proj = u1_projector(n,3) 
print(proj.shape[0])  

eff_ham = proj @ ham @ proj.conj().T
target_evals = np.linalg.eigvalsh(eff_ham)

#ansatz_func = one_local_ansatz
#bounds = [(-4, 4)] * (n+1)  # need to be expanded
ansatz_func = nn_ansatz
bounds = [(-2, 2)] * (2*n)  # need to be expanded
print(len(bounds)) 
quiet=True

last_print_time = time.time()
result = dual_annealing(lambda params: hungarian_cost(params, target_evals, n, ansatz_func, quiet), bounds)

hungarian_cost(result.x, target_evals, n, ansatz_func, quiet=False)

20
12
Cost: 2.9795631387720016
Cost: 3.7522608572142846
Cost: 3.1894395608322035
Cost: 2.3143374697586925
Cost: 3.3471735479677363
Cost: 2.1633172440307695
Cost: 0.5626008391821458
Cost: 5.888126894846916
Optimal Assignments:
Worker 1 → Task 11 (Cost: 0.00030913152251099696)
Worker 2 → Task 19 (Cost: 0.004224612919776938)
Worker 6 → Task 18 (Cost: 8.815140359885731e-05)
Worker 7 → Task 14 (Cost: 0.0005075423279365943)
Worker 16 → Task 12 (Cost: 6.264001407407793e-05)
Worker 19 → Task 7 (Cost: 0.17171244165119848)
Worker 22 → Task 10 (Cost: 2.3705949127178494e-05)
Worker 23 → Task 4 (Cost: 0.0004430968734685514)
Worker 25 → Task 2 (Cost: 0.004799194079445179)
Worker 26 → Task 15 (Cost: 0.00026544963716368386)
Worker 27 → Task 9 (Cost: 0.00015394128717899358)
Worker 28 → Task 17 (Cost: 0.00018717198527928858)
Worker 32 → Task 16 (Cost: 0.004713575886991173)
Worker 35 → Task 13 (Cost: 7.137748327945559e-05)
Worker 48 → Task 3 (Cost: 0.0005777436210694376)
Worker 55 → Task 0 (Cost: 7.20067

0.19345263146813815

In [33]:
target_evals 

array([-6.98791841, -5.20775094, -3.60387547, -3.60387547, -2.49395921,
       -2.49395921, -2.        , -0.89008374, -0.89008374, -0.21983253,
        0.21983253,  0.89008374,  0.89008374,  2.        ,  2.49395921,
        2.49395921,  3.60387547,  3.60387547,  5.20775094,  6.98791841])

## XXZ model

In [17]:
# testing NN ansatz with Hungarian algorithm + simulated annealing
n = 5
ham = xxz_ham(n)
proj = u1_projector(n,2) 
print(proj.shape[0])  

eff_ham = proj @ ham @ proj.conj().T
target_evals = np.linalg.eigvalsh(eff_ham)

ansatz_func = nn_ansatz
bounds = [(-1, 1)] * (2 * n)  # need to be expanded
print(len(bounds)) 
quiet=True

last_print_time = time.time()
result = dual_annealing(lambda params: hungarian_cost(params, target_evals, n, ansatz_func, quiet), bounds)

hungarian_cost(result.x, target_evals, n, ansatz_func, quiet=False)

10
10
Cost: 0.03724477697744134
Cost: 2.3747850400494093
Cost: 3.8956419047811295
Optimal Assignments:
Worker 1 → Task 6 (Cost: 3.8032238552920816e-08)
Worker 2 → Task 8 (Cost: 6.883908283850104e-05)
Worker 5 → Task 9 (Cost: 0.006602111405924482)
Worker 9 → Task 1 (Cost: 2.2790452325338606e-07)
Worker 16 → Task 4 (Cost: 0.0026632550998686444)
Worker 19 → Task 3 (Cost: 3.02438474264477e-09)
Worker 20 → Task 7 (Cost: 0.01212207255175346)
Worker 25 → Task 0 (Cost: 0.009510228872446014)
Worker 26 → Task 2 (Cost: 0.0025856320373778274)
Worker 28 → Task 5 (Cost: 1.0608901188091657e-08)

Total Minimum Cost: 0.033552418620256666


0.033552418620256666

In [21]:
# testing two-local ansatz with Hungarian algorithm + simulated annealing
n = 6
ham = xxz_ham(n)
proj = u1_projector(n,3) 
print(proj.shape[0])  

eff_ham = proj @ ham @ proj.conj().T
target_evals = np.linalg.eigvalsh(eff_ham)

ansatz_func = two_local_ansatz
bounds = [(-5, 5)] * (n + 1 + int(n*(n-1)/2))
print(len(bounds)) 
quiet=True

last_print_time = time.time()
result = dual_annealing(lambda params: hungarian_cost(params, target_evals, n, ansatz_func, quiet), bounds)

hungarian_cost(result.x, target_evals, n, ansatz_func, quiet=False)

20
22
Cost: 0.4418724781829984
Cost: 2.0424634298163156
Cost: 0.26015901570066574
Cost: 8.462257227577451
Cost: 16.53909740803774
Cost: 6.070875353021597
Cost: 17.030109069661755
Cost: 7.643102796626452
Cost: 5.049878092816561
Cost: 16.258769051284318
Cost: 5.8484467036040035
Cost: 28.572171342700514
Cost: 7.170617133651238
Cost: 10.368949031320785
Cost: 5.3018302001188164
Cost: 17.219806439974043
Cost: 13.223656411209125
Cost: 4.681321268031265
Cost: 6.4549837952607785
Cost: 16.56673856777312
Cost: 21.568749505439648
Cost: 10.243655433189888
Cost: 1.7924908731159777
Cost: 12.426817991990738
Cost: 11.224337416997177
Cost: 0.5663217527580267
Cost: 0.4220915095237057
Cost: 20.822467747222333
Optimal Assignments:
Worker 6 → Task 11 (Cost: 0.00025843841672068546)
Worker 9 → Task 5 (Cost: 0.0008420206212145231)
Worker 13 → Task 0 (Cost: 0.00811692109021589)
Worker 14 → Task 19 (Cost: 0.03406201536933251)
Worker 19 → Task 9 (Cost: 2.2432035051034305e-05)
Worker 27 → Task 17 (Cost: 0.00607561

0.09257402703579648

## XXX model

In [132]:
# Can I generate subspaces with fixed total S and total Sz
n = 10
j = 1
m = 1    

Sz = total_sz(n)
S_sq = total_spin_squared(n)
P, basis = projector(n,j,m)
for i in range(basis.shape[1]): 
    print(basis[:,i].conj().T @ S_sq @ basis[:,i], basis[:,i].conj().T @ Sz @ basis[:,i]) 

print(basis.shape[1]) 

(8.000000000000002+0j) (2.000000000000001+0j)
(8.000000000000018+0j) (2.0000000000000044+0j)
(8.000000000000004+0j) (2.0000000000000013+0j)
(7.999999999999995+0j) (1.9999999999999996+0j)
(8+0j) (2+0j)
(7.999999999999992+0j) (1.9999999999999982+0j)
(8.00000000000001+0j) (2.000000000000003+0j)
(7.999999999999992+0j) (1.9999999999999982+0j)
(8.000000000000007+0j) (2.0000000000000013+0j)
(7.999999999999997+0j) (1.9999999999999987+0j)
(7.999999999999999+0j) (1.9999999999999991+0j)
(7.999999999999995+0j) (1.999999999999999+0j)
(7.999999999999997+0j) (1.9999999999999993+0j)
(7.999999999999999+0j) (2+0j)
(8+0j) (1.9999999999999998+0j)
(8.000000000000012+0j) (2.0000000000000027+0j)
(7.99999999999997+0j) (1.9999999999999925+0j)
(8.000000000000012+0j) (2.0000000000000027+0j)
(8.000000000000014+0j) (2.0000000000000036+0j)
(8.00000000000001+0j) (2.000000000000002+0j)
(7.99999999999997+0j) (1.9999999999999925+0j)
(8.000000000000012+0j) (2.000000000000003+0j)
(8.000000000000005+0j) (2.000000000000001

In [ ]:
# searching over all permutations with nn_ansatz
eff_ham = basis.conj().T @ xxx_ham(n) @ basis
target_evals = np.linalg.eigvalsh(eff_ham)

random_perms = [np.random.permutation(target_evals) for _ in range(100)]
for perm in tqdm(random_perms):     
    params = np.random.rand(2*n) 
    res = minimize(nn_cost, params, args=(n, perm), method='Powell')
    print(nn_cost(res.x, n, target_evals)) 

In [ ]:
# testing two-local ansatz
def create_callback(n, target_evs):
    def callback(xk):
        current_cost = two_local_cost(xk, n, target_evs)
        print(f"Current cost: {current_cost}")
    return callback

initial_params = np.random.rand(n+1 + int(n*(n-1)/2))

result = minimize(
    two_local_cost, 
    initial_params, 
    args=(n, target_evals),  # Pass additional variables to the cost function
    method='Powell', 
    callback=create_callback(n, target_evals)  # Pass closure to include extra variables
)
print(result.fun) # Final cost 

In [112]:
# testing Hungarian algorithm + simulated annealing with nn_ansatz
n = 8
j = 1
m = 1  

Sz = total_sz(n)
S_sq = total_spin_squared(n)
P, basis = projector(n,j,m)

print(basis.shape[1]) 
eff_ham = basis.conj().T @ xxx_ham(n) @ basis
target_evals = np.linalg.eigvalsh(eff_ham)

ansatz_func = nn_ansatz
bounds = [(-1, 1)] * (2 * n)  # need to be expanded
quiet=True

last_print_time = time.time()
result = dual_annealing(lambda params: hungarian_cost(params, target_evals, n, ansatz_func, quiet), bounds)

hungarian_cost(result.x, target_evals, n, ansatz_func, quiet=False)

Cost: 0.09262098581960737
Optimal Assignments:
Worker 5 → Task 25 (Cost: 0.003770667097336089)
Worker 6 → Task 19 (Cost: 1.784267581592114e-05)
Worker 7 → Task 21 (Cost: 0.0022501796005280905)
Worker 9 → Task 8 (Cost: 0.001649771949137513)
Worker 15 → Task 16 (Cost: 5.2019330509800454e-08)
Worker 31 → Task 6 (Cost: 3.196889697565375e-05)
Worker 39 → Task 3 (Cost: 3.4052183295329996e-10)
Worker 53 → Task 26 (Cost: 0.002384016764500174)
Worker 54 → Task 20 (Cost: 0.0015608858789831759)
Worker 61 → Task 10 (Cost: 0.0008266220733800234)
Worker 72 → Task 7 (Cost: 0.006688411924830051)
Worker 74 → Task 1 (Cost: 0.0010917730314448093)
Worker 79 → Task 2 (Cost: 0.00020987917019121838)
Worker 81 → Task 4 (Cost: 0.0015706869454252814)
Worker 109 → Task 0 (Cost: 0.006338038851148475)
Worker 124 → Task 18 (Cost: 0.0017261810266968325)
Worker 130 → Task 22 (Cost: 1.596253462743391e-09)
Worker 149 → Task 27 (Cost: 0.022595465715853802)
Worker 160 → Task 11 (Cost: 0.0002626121957685079)
Worker 168 → 

0.09262098581960737

In [210]:
# testing Hungarian algorithm + simulated annealing with two_local ansatz
n = 10
j = 2
m = 2   

Sz = total_sz(n)
S_sq = total_spin_squared(n)
P, basis = projector(n,j,m)

print(basis.shape[1]) 
eff_ham = basis.conj().T @ xxx_ham(n) @ basis
target_evals = np.linalg.eigvalsh(eff_ham)

ansatz_func = two_local_ansatz
bounds = [(-1, 1)] * (n + 1 + int(n*(n-1)/2))  # need to be expanded
print(len(bounds))
quiet=True

last_print_time = time.time()
result = dual_annealing(lambda params: hungarian_cost(params, target_evals, n, ansatz_func, quiet), bounds)

hungarian_cost(result.x, target_evals, n, ansatz_func, quiet=False)

75
56
Cost: 0.42288255372305394
Cost: 0.3967059803962218
Cost: 0.45729154608396755
Cost: 0.45833032406636964
Cost: 0.41805459806695766
Cost: 0.3979562861511392
Cost: 0.5731431682058169
Cost: 0.42025442501184107
Cost: 0.5215228086617409
Cost: 0.40889933586704097
Cost: 0.40103917719638094
Cost: 0.4058964376747088
Cost: 0.3386527858589832
Cost: 0.3386527458589834
Cost: 0.3386528458589828
Cost: 0.3386526658589839
Cost: 0.3386528658589827
Cost: 0.33865272585898354
Cost: 2.6289778178152994
Cost: 2.6289775678152796
Cost: 2.6289775278152994
Cost: 2.628977847815292
Cost: 2.6289780278152928
Cost: 2.6289779278152987
Cost: 2.6289777478152985
Cost: 0.4544300489411879
Cost: 0.45442996894118837
Cost: 0.4544299689411879
Cost: 0.45443000894119034
Cost: 0.4544298689411945
Cost: 0.45443012894119705
Cost: 0.4160336066605718
Cost: 0.4160334666605749
Cost: 0.41603368666057133
Cost: 0.4160335066605724
Cost: 0.41603346666057334
Cost: 0.4160336066605755
Cost: 0.5113190007496802
Cost: 0.5113190307496787
Cost: 0

KeyboardInterrupt: 

In [189]:
# testing Hungarian algorithm + simulated annealing with two_local ansatz on full XXX Hamiltonian
n = 4

target_evals = np.linalg.eigvalsh(xxx_ham(n))
print(len(target_evals))

ansatz_func = two_local_ansatz
bounds = [(-2, 2)] * (n + 1 + int(n*(n-1)/2))  # need to be expanded
#bounds = [(-2,2)] * (n+1)
print(len(bounds))
quiet=True

last_print_time = time.time()
#result = dual_annealing(lambda params: hungarian_cost(params, target_evals, n, ansatz_func, quiet), bounds)

#hungarian_cost(result.x, target_evals, n, ansatz_func, quiet=False)

16
11


In [190]:
target_evals 

array([-1.6160254 , -0.95710678, -0.95710678, -0.95710678, -0.25      ,
       -0.25      , -0.25      ,  0.1160254 ,  0.45710678,  0.45710678,
        0.45710678,  0.75      ,  0.75      ,  0.75      ,  0.75      ,
        0.75      ])

In [200]:
# testing Hungarian algorithm + genetic algorithm with two_local ansatz
n = 5
j = 1/2
m = 1/2   

Sz = total_sz(n)
S_sq = total_spin_squared(n)
P, basis = projector(n,j,m)

eff_ham = basis.conj().T @ xxx_ham(n) @ basis
target_evals = np.linalg.eigvalsh(eff_ham)
print(target_evals) 

[-1.92788625 -1.20710678 -0.66081859  0.20710678  0.58870484]


In [191]:
n = 4
target_evals = np.linalg.eigvalsh(xxx_ham(n))
print(target_evals)

-0.25, 0.75
-0.95710678, 0.45710678
1.6160254, 0.1160254


[-1.6160254  -0.95710678 -0.95710678 -0.95710678 -0.25       -0.25
 -0.25        0.1160254   0.45710678  0.45710678  0.45710678  0.75
  0.75        0.75        0.75        0.75      ]


In [192]:
n = 4
target_evals = np.linalg.eigvalsh(xxz_ham(n))
print(target_evals)

-3.462172
-3.06155281, 1.06155281
-1, 3
-0.65138782, 1.15138782, -1.5962912, 1.0962912
-0.60753298, 1.06970498

[-3.462172   -3.06155281 -1.5962912  -1.5962912  -1.         -0.65138782
 -0.65138782 -0.60753298  1.06155281  1.06970498  1.0962912   1.0962912
  1.15138782  1.15138782  3.          3.        ]


In [196]:
n = 4
target_evals = np.linalg.eigvalsh(xy_ham(n))
print(target_evals)

x = 4.47213595
y = 1.23606798
z = 2
np.array([-x, -y-z, -y-z, -z, -y, -y, z-z, z-z, z-z, z-z, y, y, z, y+z, y+z, x]) - target_evals


[-4.47213595 -3.23606798 -3.23606798 -2.         -1.23606798 -1.23606798
  0.          0.          0.          0.          1.23606798  1.23606798
  2.          3.23606798  3.23606798  4.47213595]


array([ 4.99957942e-09, -2.50021026e-09, -2.50021026e-09, -4.44089210e-16,
       -2.50021026e-09, -2.50021026e-09,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.50021026e-09,  2.50021026e-09,
       -1.33226763e-15,  2.50020982e-09,  2.50020982e-09, -4.99957853e-09])

# Old

In [ ]:
n = 4
Sz = total_sz(n)
S_sq = total_spin_squared(n)
U = np.loadtxt(f'N={n}.txt', delimiter=',')

In [ ]:
j = 1
m = -1

j_cur = 0 if n % 2 == 0 else 1/2
sector_start = 0 
while j_cur < j: 
    sector_start += su2_multiplicity(n,j_cur) * su2_dimension(j_cur)
    j_cur += 1

sector_end = sector_start + su2_multiplicity(n,j) * su2_dimension(j)
m_offset = int((-m * 2 + 2*j)/2) if (m*2)%2 == 1 else (-m+j)

indices = []
for i in range(su2_multiplicity(n,j)):
    indices.append(sector_start + i*su2_dimension(j) + m_offset)

sector_start, sector_end, indices 

(2, 11, [4, 7, 10])

In [ ]:
for i in indices: 
    print(U[:,i].conj().T @ S_sq @ U[:,i], U[:,i].conj().T @ Sz @ U[:,i])

(8.000000000000004+0j) (-2.0000000000000018+0j)
(8.000000000000012+0j) (-2.0000000000000027+0j)
(7.999999999999998+0j) (-1.9999999999999998+0j)


In [ ]:
ham = xxx_ham(10)
evals, _ = np.linalg.eigh(ham) 

In [ ]:
spectrum_list = []
spins = [0,1,2,3,4,5]
for spin in spins:
    with open(f'Q=10_S={spin}.txt', 'r') as file:
        spectrum = [float(line.strip()) for line in file.readlines()]
        spectrum_list.append(spectrum) 

sum([len(spectrum)*(2*spin+1) for spectrum,spin in zip(spectrum_list,spins)])  

1024

In [ ]:
full_spectrum = np.concatenate([spectrum*(2*spin+1) for spectrum,spin in zip(spectrum_list, spins)]) 
full_spectrum = np.sort(full_spectrum) 
np.allclose(evals, full_spectrum)

True